## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Исходная таблица содержит следующие данные:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

### Шаг 1. Импорт библиотек, загрузка данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_credit.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

При первом ознакомлении с таблицей можно сказать, что в двух столбцах, 'days_employed' и 'total_income', отсутствует часть данных. Вместо 21525 строк в них заполнены только 19351 (количество строк с пропусками — 2174). Также можно отметить, что многие значения в столбце 'days_employed' приведены некорректно: некоторые числа — отрицательные (что невозможно для количества отработанных дней), часть превышает разумно допустимые значения. Так, 343937 дней — это более 942-х лет. К счастью, трудовой стаж пока что не измеряется столетиями. Тип данных int64 лучше бы подошел столбцу 'days_employed', чем присущий на данный момент float64, так как количество отработанных дней — по определению целое число.

Скорее всего, проблемы, связанные обоими столбцами имеют технический характер. Необходимо обратиться к коллегам, которые отвечают за сбор и хранение информации, чтобы предупредить о существующих сложностях и предотвратить их появление в будущем.

### Шаг 2. Предобработка данных

### Обработка пропусков

Прежде чем удалить строки с пропущенными значениями в столбце 'total_income', убедимся, что пропуски не связаны с полным отсутствием дохода у этих заемщиков и нули не были по ошибке заменены значением NaN. Для этого посчитаем количество уникальных значений в столбце 'income_type'. 

In [4]:
print(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


За исключением категории "компаньоны", людей, доход которых мог бы равняться нулю, в таблице всего 5: двое безработных, два предпринимателя и один студент. Компаньонов 3837 человек. Маловероятно, что 57% компаньонов имеет нулевой доход и во всех этих случаях данные были пропущены, причем только для этой категории заемщиков, ведь прочие значения в столбце 'total_income' указаны верно.

Все остальные заемщики должны иметь доход, отличный от нуля. Значит, пропуски в столбце 'total_income' не связаны с их родом деятельности и это не просто замена нуля значением NaN. Можем смело удалить столбец 'days_employed', т. к. в анализе он все равно не учитывается. Таким образом не придется уменьшать количество наблюдений. Пропуски в 'total_income' заменяем на медианное значение, т. к. этому столбцу присущ количественный тип данных.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)
df.dropna(axis = 'columns', inplace = True)
df.isna().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Проверяем количество пропусков с помощью комбинации методов isna() и sum(). Пропуски замечены только в двух столбцах. В обоих столбцах количество пропущенных значений равно 2174. 21525 (общее количество строк в таблице)  минус 19351 (количество заполненных строк в столбцах 'days_employed' и 'total_income') как раз и дадут 2174, количество ячеек с типом данных NaN.

Столбец total_income содержит количественные данные, поэтому недостающие значения можно заменить на медиану (т. к. доход может иметь значительные разброс, среднее арифметическое здесь не подойдет). Так как зависимость между трудовым стажем и возвратом кредита в срок выяснять не нужно, столбец 'days_employed' можно просто удалить. Проверяем таблицу на наличие пропусков с помощью все тех же isna() и sum(). Результат — пропусков не осталось.

### Замена типа данных

In [7]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int64 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int32 
 10  purpose           21525 non-null  object
dtypes: int32(1), int64(5), object(5)
memory usage: 1.7+ MB


### Вывод

В изменении типа данных нуждается только столбец 'total_income'. Так как числа, которые в нем фигурируют, достаточно большие, необходимость в десятичных знаках отсутствует. Изменим тип данных в этом столбце на int64. Делаем это с помощью функции astype(), так как функция to_numeric(), которая меняет тип данных в ячейках на вещественный, здесь не подходит. Проверим.

### Обработка дубликатов / устранение ошибок

In [8]:
print(df.duplicated().sum())

54


In [9]:
df = df.drop_duplicates().reset_index()
print(df.duplicated().sum())
print(df.info())

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             21471 non-null  int64 
 1   children          21471 non-null  int64 
 2   dob_years         21471 non-null  int64 
 3   education         21471 non-null  object
 4   education_id      21471 non-null  int64 
 5   family_status     21471 non-null  object
 6   family_status_id  21471 non-null  int64 
 7   gender            21471 non-null  object
 8   income_type       21471 non-null  object
 9   debt              21471 non-null  int64 
 10  total_income      21471 non-null  int32 
 11  purpose           21471 non-null  object
dtypes: int32(1), int64(6), object(5)
memory usage: 1.9+ MB
None


Проверяем таблицу на наличие грубых дубликатов. Их в таблице 54 штуки. Возможные причины их появления — технические ошибки или повторный ввод данных одних и тех же заемщиков персоналом. Удалим методом drop_duplicates(), так как он лучше всего подходит для удаления грубых дубликатов, и сбросим индексы методом reset_index(). Проверяем: дубликаты удалены. Теперь во всех столбцах по 21471 строке.

Проверяем таблицу на наличие прочих дубликатов, неточностей и ошибок.

In [10]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
df['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

В таблице 76 строк, в которых количество детей в семье равно 20. Скорее всего, это ошибка, т. к. семей, в которых от 6 до 19, мы не наблюдаем. К тому же, это значение кажется крайне маловероятным. Заменяем 20 на 2 как наиболее вероятный вариант. Также можно заметить, что в 47 случаях количество детей указано как -1. Очевидно, что это тоже ошибка. Заменяем на 1.

Как и раньше, сообщаем об ошибке ответственным за сбор и хранение информации коллегам.

In [12]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В столбце 'education' проблема другого характера. Одни и те же данные внесены в таблицу в разном регистре, что в будущем повлияет на результаты группировки. Хоть этот столбец и не учитывается в анализе, неплохо было бы его подготовить для дальнейшей работы. Приводим все значения в столбце к нижнему регистру с помощью функции str.lower().

In [14]:
df['education_id'].value_counts()

1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

Количество записей каждого типа в столбцах 'education' и 'education_id' совпадает. Значит, текстовые и цифровые обозначения уровня образования совпадают и между двумя столбцами нет противоречий.

In [15]:
print(df['family_status'].value_counts())
print()
print(df['family_status_id'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64


In [16]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Количество записей каждого типа в столбцах 'family_status' и 'family_status_id' также совпадает. Значит, с группировкой этих значений проблем возникнуть не должно. Единственная проблема, с который мы столкнулись — ненужная капитализация в строках со значением "Не женат / не замужем". На результаты исследования это не повлияет, но негативно скажется на единообразии данных и читаемости таблицы как таковой. Исправляем с помощью метода str.lower(), проверяем.

In [17]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

В столбце 'gender' есть ошибка: в одной из строк неверно проставлен пол. Выяснять зависимость возврата средств по кредиту от пола не нужно, но эти данные могут пригодиться в будущем. Ошибка закралась только в одну строку, а это менее 0.005% всех наблюдений. Удаляем строку, содержащую ошибочную информацию. Новое количество строк в таблице — 21470.

In [18]:
df = df.drop(df[df['gender'] == 'XNA'].index)
df['gender'].value_counts()

F    14189
M     7281
Name: gender, dtype: int64

In [19]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5079
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Посмотрим на данные в столбце 'income_type', отвечающий за источник дохода. Не представляется возможным установить, должны ли предприниматели попасть в ту же категорию, что и компаньоны. Вполне возможно, что компаньоны — это люди, совместно ведущие бизнес с кем-то еще, а предприниматели — его единоличные владельцы. Если это действительно так, то и оценивать их следует по-разному. К безработным также можно было бы отнести студентов, но, возможно, их специально выделили в отдельную группу. Оставляем столбец без изменений, т. к. влияние типа дохода на возврат средств оценивать не нужно. Однако следует обратиться к сотрудникам, ответственным за распределение клиентов банка по типу дохода (или соответствующей документации) с целью выяснения подробностей.

In [20]:
df['debt'].value_counts()

0    19729
1     1741
Name: debt, dtype: int64

Со столбцом 'debt' все в порядке.

In [21]:
print(df['total_income'].sort_values().head(10))
print(df['total_income'].sort_values().tail(10))

14562    20667
12989    21205
16147    21367
1598     21695
14254    21895
10869    22472
18472    23844
9064     24457
10059    25227
12038    25308
Name: total_income, dtype: int32
11058    1286280
15242    1350245
18317    1427934
18332    1551152
17471    1597613
17147    1711309
20759    1715018
9163     1726276
19562    2200852
12396    2265604
Name: total_income, dtype: int32


Чтобы удостовериться в корректности данных в столбце 'total_income', выведем на экран по 10 первых и последних значений. Видим, что аномалий не наблюдается.

In [22]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Оценим уникальные значения в столбце 'purpose'. Несмотря на различный формат записи, можно заметить, что все они касаются 1) проведения свадьбы, 2) операций с недвижимостью, 3) покупки автомобиля, 4) получения образования. Для упорядочивания и категоризации наблюдений в данном столбце прибегнем к лемматизации.

### Вывод

В исходных данных обнаружилось небольшое количество грубых дубликатов, которые были успешно удалены. Многие столбцы содержали ошибки, неточности или некорректные формы записи. Столбец 'days_employed' был удален как содержащий слишком большое количество ошибок и неточностей, причины которых и возможные варианты замены невозможно установить. Данные в остальных столбцах были приведены к единому формату, очищены от пропусков и некорректных значений. Также было проверено соответствие между столбцами с текстовыми и цифровыми значениями, отвечающими за один и тот же параметр (например 'family_status' и 'family_status_id').

### Лемматизация

In [30]:
from pymystem3 import Mystem
m = Mystem()

def lemmatizator(i):
    lemmas = m.lemmatize(i)
    return lemmas

In [31]:
df['purpose_lemmas'] = df['purpose'].apply(lemmatizator)

df.head(10)

KeyboardInterrupt: 

### Вывод

Лемматизация позволит нивелировать разницу в написании целей кредита разными способами и разбить их на соответствующие категории, которые в последствии будет легче анализировать. В рамках задачи лемматизации в таблице создается новый столбец 'purpose_lemmas', в который добавляются лемматизированные значения столбца 'purpose'. Процедура проводится при помощи библиотеки pymystem3, а также методов lemmatize() и apply().

### Категоризация данных

В категоризации нуждаются два из интересующих нас столбцов, но по разным причинам. В столбце 'purpose' всего четыре вида причин, но многие из них записаны по-разному. В столбце 'total_income' слишком большое количество уникальных значений. Всех людей придется разбить на категории по уровню дохода.

Сначала разберемся с разбиением целей кредита по группам. Назначим числовые значения для каждой из четырех категорий целей кредита, сохраним их в новом столбце.

In [ ]:
def lemmas_to_numbers(i):
    if 'свадьба' in i:
        return 1
    elif 'автомобиль' in i:
        return 2
    elif 'недвижимость' in i or 'жилье' in i:
        return 3
    elif 'образование' in i:
        return 4
    else:
        return 9

df['purpose_id'] = df['purpose_lemmas'].apply(lemmas_to_numbers)
df['purpose_id'].value_counts()

Проверим, все ли строки получили нужный идентификатор, добавив в функцию lemmas_to_numbers дополнительное условие. Как видим, ни в одной строке нет значения 9, которое говорило бы о неучтенных категориях. Больше всего кредитов берут на приобретение недвижимости, на втором месте — покупка автомобиля, потом — получение образования, и лишь потом свадьба.

Теперь удостоверимся в том, что количество строк с ненулевыми значениями в столбце 'purpose_id' такое же, как и во всей остальной таблице.

In [ ]:
df.info()

Разобьем всех заемщиков на квартили по уровню доходу. Для этого применим метод describe() и сравним содержащиеся в столбце 'total_income' значения со значениями, которые соответствуют 25%, 50% и 75%. На основании этого распределим заемщиков по категориям. Затем удостоверимся в том, что количество строк с ненулевыми значениям в столбце 'income_lev' такое же, как и во всей остальной таблице.

In [ ]:
print(df['total_income'].describe().astype(int))

In [ ]:
def income_level(i):
    if i <= 107652:
        return 'низкий'
    elif 107652 < i <= 145017:
        return 'средний'
    elif 145017 < i <= 195750:
        return 'выше среднего'
    elif i > 195750:
        return 'высокий'
    else:
        return 9

df['income_lev'] = df['total_income'].apply(income_level)
print(df['income_lev'].value_counts())
print()
print(df.info())

### Вывод

Из всех переменных, связь которых с возвратом кредита нужно установить, только столбец 'purpose' нуждался в лемматизации. В категоризации нуждались два столбца: 'purpose' и 'total_income'. В рамках этой задачи мы создали три новых столбца. Один — отвечающий за лемматизированные значения столбца 'purpose'. Второй — за категорию, к которой категории должно принадлежать наблюдение в зависимости от значения столбца 'purpose'. Третий — за категорию, связанную с уровнем дохода заемщика.

Можем перейти к непосредственному анализу связи некоторых характеристик заемщика и вероятности возвращения им кредита в срок.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_result = df.groupby('children').agg({'debt': ['count', 'sum',]})
children_result['scoring'] = (children_result['debt']['sum'] / children_result['debt']['count']) * 100
print(children_result)
print()

children_count = children_result.loc[1:, ('debt', 'count')].sum()
children_sum = children_result.loc[1:, ('debt', 'sum')].sum()
children_scoring = (children_sum / children_count)
print('Процент людей с детьми, у которых есть задолженность: {:.1%}.'.format(children_scoring))

### Вывод

Создаем новую таблицу, состоящую из следующих столбцов:
1. количество заемщиков в каждой из категорий (в данном случае категории — количество детей),
2. количество заемщиков в каждой категории,
3. сумма значений столбца 'debt' из основной таблицы для каждой категории замещиков.

Для того, чтобы выяснить, есть ли зависимость между количеством детей и возвратом средств в срок, разделим сумму всех значений столбца 'debt' каждой категории заемщиков на общее количество заемщиков в соответствующей категории и умножим на 100, чтобы получить процент людей, задержавших выплату. Впоследствии этот же метод будем применять и к другим параметрам.

- 7,5% бездетных людей не вернули кредит в срок.
- 9,2% людей с одним ребенком не вернули кредит в срок.
- 9,5% людей с двумя детьми не вернули кредит в срок.
- 8,2% людей с тремя детьми не вернули кредит в срок.
- 9,8% людей с четырьмя детьми не вернули кредит в срок. Однако в данной категории всего 41 наблюдение, говорить о репрезентативности такой выборки сложно.
- Все люди, в семье которых пятеро детей, вернули деньги в срок. То же, что и для предыдущей категории. Выборка еще меньше, репрезентативным этот результат считаться не может.

Для того, чтобы узнать, как зависит возврат денежных средств в срок от наличия / отсутствия детей, просуммируем данные всех заемщиков, у которых есть дети, и рассчитаем, какой процент из них не возвращает средства в срок.

- 9,2% людей с детьми не вернули кредит в срок.
- 7,5% бездетных людей не вернули кредит в срок.

Если не считать семьи с пятью детьми, то бездетные люди чаще возвращают средства в срок. Самый высокий процент людей, не вернувших кредит в срок (с количеством наблюдений больше 50), — у заемщиков с двумя детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
famstatus_result = df.groupby('family_status').agg({'debt': ['count', 'sum',]})
famstatus_result['scoring'] = (famstatus_result['debt']['sum'] / famstatus_result['debt']['count']) * 100
print(famstatus_result.sort_values(by='scoring'))

### Вывод

Рассчитываем зависимость возврата средств от семейного положения тем же способом (разве что без объединения нескольких категорий в одну). Получаем следующей результат:

- 6,6% вдов и вдовцов не вернули кредит в срок.
- 7,1% людей, состоящих в разводе, не вернули кредит в срок.
- 7,5% женатых / замужних людей не вернули кредит в срок.
- 9,3% людей, проживающих в гражданском браке, не вернули кредит в срок.
- 9,8% неженатых / незамужних людей не вернули кредит в срок.

Вдовы, вдовцы и разведенные люди реже всего задерживают выплаты по кредитам. Чаще всего задерживают выплаты по кредиту неженатые / незамужние люди, а также люди, живущие в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_result = df.groupby('income_lev').agg({'debt': ['count', 'sum',]})
income_result['scoring'] = (income_result['debt']['sum'] / income_result['debt']['count']) * 100
print(income_result)

### Вывод

Уровень дохода также влияет на то, какой процент заемщиков возвращает деньги в срок.

- 8% людей с низкими уровнем дохода не вернули кредит в срок.
- 8,5% людей со средним уровнем дохода не вернули кредит в срок.
- 8,9% людей с уровнем дохода выше среднего не вернули кредит в срок.
- 7,1% людей с высоким уровнем дохода не вернули кредит в срок.

Люди, попавшие в топ-25 процентов по уровню доходов, реже остальных задерживают выплаты по кредитам, в то время как люди с уровнем дохода выше среднего делают это чаще всего.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_result = df.groupby('purpose_id').agg({'debt': ['count', 'sum',]})
purpose_result['scoring'] = (purpose_result['debt']['sum'] / purpose_result['debt']['count']) * 100
print(purpose_result)

df_pivot = purpose_result.pivot_table(index=['purpose_id'], columns=('debt''source'), values='visits', aggfunc='sum')

### Вывод

Как и все предыдущие параметры, цель кредита также влияет на шанс уплаты по кредиту в срок.

- 8% людей, взявших кредит на свадьбу, не вернули его в срок.
- 9,4% людей, взявших кредит для приобретения автомобиля, не вернули его в срок.
- 7,2% людей, которым кредит понадобился для приобретения недвижимости, не вернули его в срок.
- 9,2% людей, взявших кредит на образование, не вернули его в срок.

Реже всего задерживают выплаты те люди, которые указали целью кредита операции с недвижимостью. Чаще всего — те, кому кредит нужен для покупки автомобиля.

### Шаг 4. Общий вывод

В ходе анализа были обнаружены пропуски, неточности и ошибки в данных, связанные с техническими сбоями или некорректной обработкой информации сотрудниками банка. О них следует известить соответствующих специалистов и устранить для повышения качества хранимой информации и последующего анализа.

Данные были распределены по категориям для удобства оценки влияния различных характеристик заемщика на ключевой показатель.

В ходе проведенного исследования удалось установить, что все четыре фактора: наличие детей, семейное положение, уровень дохода и цель кредита — влияют на вероятность задержки выплаты по кредиту. Данные могут быть использованы для принятия решений о выдаче кредита тем или иным категориям граждан, а также для расчета риска невыплаты кредита в срок.

*Какие выводы может сделать аналитик из этого задания?*
- Предобработка данных занимает куда больше времени, чем непосредственный анализ интересующих данных.
- В сырых данных встречается большое количество ошибок, пропусков и неточностей, не отсеяв которые невозможно получить точный результат.
- Ввиду предыдущего пункта аналитик должен особое внимание уделять обратной связи, сообщая коллегам о возникающих проблемах. Так, последних можно будет избежать в будущем, тем самым сократив время на предобработку данных и снизив количество потенциальных ошибок в выводах.
